In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, DenseVector
from pyspark.mllib.clustering import KMeans, KMeansModel
import numpy
import os
import sys
from numpy import array
from math import sqrt

sc =SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/08 08:19:43 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


### K-means implementing MLib
Code example is obtained from: https://spark.apache.org/docs/1.5.1/mllib-clustering.html

In [5]:
# os.environ['PYSPARK_PYTHON'] = sys.executable
# Load and parse the data
data = sc.textFile("/home/ubuntu/protein/sampled_embeddings_reduced.csv")
parsedData = data.map(lambda line: array([float(x) for x in line.split(',')[1:]]))

# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 2, maxIterations=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

# Save and load model
clusters.save(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")
sameModel = KMeansModel.load(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")

23/01/08 08:38:20 WARN TaskSetManager: Lost task 0.0 in stage 2.0 (TID 23) (slave1 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/ubuntu/spark/python/pyspark/worker.py", line 668, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/home/ubuntu/spark/python/pyspark/worker.py", line 85, in read_command
    command = serializer._read_with_length(file)
  File "/home/ubuntu/spark/python/pyspark/serializers.py", line 173, in _read_with_length
    return self.loads(obj)
  File "/home/ubuntu/spark/python/pyspark/serializers.py", line 471, in loads
    return cloudpickle.loads(obj, encoding=encoding)
  File "/home/ubuntu/spark/python/pyspark/mllib/__init__.py", line 26, in <module>
    import numpy
ModuleNotFoundError: No module named 'numpy'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.api.python.PythonRun

Py4JJavaError: An error occurred while calling o70.trainKMeansModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 2.0 failed 4 times, most recent failure: Lost task 3.3 in stage 2.0 (TID 33) (slave1 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/ubuntu/spark/python/pyspark/worker.py", line 668, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/home/ubuntu/spark/python/pyspark/worker.py", line 85, in read_command
    command = serializer._read_with_length(file)
  File "/home/ubuntu/spark/python/pyspark/serializers.py", line 173, in _read_with_length
    return self.loads(obj)
  File "/home/ubuntu/spark/python/pyspark/serializers.py", line 471, in loads
    return cloudpickle.loads(obj, encoding=encoding)
  File "/home/ubuntu/spark/python/pyspark/mllib/__init__.py", line 26, in <module>
    import numpy
ModuleNotFoundError: No module named 'num